In [ ]:
import warnings
warnings.filter

In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow
from tensorflow import keras
from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,BatchNormalization
from keras import initializers
from keras.utils.np_utils import to_categorical
from keras.losses import CategoricalCrossentropy

c:\Users\admin\anaconda3\envs\keras2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#### 옵티마이저들 적재(주로 이정도만씀)
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adagrad,Nadam

In [4]:
from sklearn.model_selection import train_test_split

In [13]:
#train_data=pd.read_clipboard()

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     888 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [15]:
## 데이터 정제
train_data=train_data.iloc[:,1:]
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  888 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


In [18]:
y=train_data.iloc[:,0].values
X=train_data.iloc[:,1:]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  888 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [ ]:
## 결측값 처리(Age,Embarked,Cabin)
# Cabin은? 결측값이 많지만 '미상'으로 처리(그게 영향있을 수 있으니까)

In [24]:
X.Cabin.value_counts()
X.Cabin.unique().size

148

In [25]:
X.Cabin.str.slice(0,1).value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [26]:
X.Cabin.fillna('N',inplace=True)
X.Cabin.str.slice(0,1)

0      N
1      C
2      N
3      C
4      N
      ..
886    N
887    B
888    N
889    C
890    N
Name: Cabin, Length: 891, dtype: object

In [27]:
X['new_cabin']=X.Cabin.str.slice(0,1)

In [29]:
X.drop(['Cabin'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        714 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   888 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [30]:
X.Embarked.value_counts()

S    644
C    168
Q     76
Name: Embarked, dtype: int64

In [31]:
X.Embarked.fillna('S',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        714 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [32]:
# 성별 평균 나이로 채우는 게 좋겠다
X.loc[X.Age.isna(),['Name','Age']]

,Name,Age
5,"Moran, Mr. James",NaN
17,"Williams, Mr. Charles Eugene",NaN
19,"Masselmani, Mrs. Fatima",NaN
26,"Emir, Mr. Farred Chehab",NaN
28,"O'Dwyer, Miss. Ellen ""Nellie""",NaN
...,...,...
859,"Razi, Mr. Raihed",NaN
863,"Sage, Miss. Dorothy Edith ""Dolly""",NaN
868,"van Melkebeke, Mr. Philemon",NaN
878,"Laleff, Mr. Kristo",NaN


In [34]:
X.Age[(X.Age.isna()) & (X.Name.str.contains('Mr.'))]= X.Age[X.Name.str.contains('Mr.')].mean()
X.Age[(X.Age.isna()) & (X.Name.str.contains('Mrs.'))]= X.Age[X.Name.str.contains('Mrs.')].mean()
X.Age[(X.Age.isna()) & (X.Name.str.contains('Miss.'))]= X.Age[X.Name.str.contains('Miss.')].mean()

C:\Users\admin\AppData\Local\Temp\ipykernel_20760\284447569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[(X.Age.isna()) & (X.Name.str.contains('Mr.'))]= X.Age[X.Name.str.contains('Mr.')].mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_20760\284447569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[(X.Age.isna()) & (X.Name.str.contains('Mrs.'))]= X.Age[X.Name.str.contains('Mrs.')].mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_20760\284447569.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [35]:
# 5명 안채워짐(아마 child)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        886 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [37]:
X.loc[X.Age.isna(),['Name','Age','Sex']]

,Name,Age,Sex
65,"Moubarek, Master. Gerios",NaN,male
159,"Sage, Master. Thomas Henry",NaN,male
176,"Lefebre, Master. Henry Forbes",NaN,male
709,"Moubarek, Master. Halim Gonios (""William George"")",NaN,male
766,"Brewe, Dr. Arthur Jackson",NaN,male


In [39]:
X.Age[X.Age.isna()]=X.Age[X.Name.str.contains('Mr')|X.Name.str.contains('Miss')|X.Name.str.contains('Mrs')].mean()

C:\Users\admin\AppData\Local\Temp\ipykernel_20760\1316724870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Age[X.Age.isna()]=X.Age[X.Name.str.contains('Mr')|X.Name.str.contains('Miss')|X.Name.str.contains('Mrs')].mean()


In [40]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     891 non-null    int64  
 1   Name       891 non-null    object 
 2   Sex        891 non-null    object 
 3   Age        891 non-null    float64
 4   SibSp      891 non-null    int64  
 5   Parch      891 non-null    int64  
 6   Ticket     891 non-null    object 
 7   Fare       891 non-null    float64
 8   Embarked   891 non-null    object 
 9   new_cabin  891 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [ ]:
# Embarked

In [ ]:
## 그냥 get_dummies 로 하면 데이터가 새로 들어왔을 때 문제가 됨
## 정수인코딩->원핫 해줘야한다

In [ ]:
# 정수인코딩 모듈 임포트
from sklearn.preprocessing import LabelEncoder

In [46]:
le_embarked=LabelEncoder()
le_embarked.fit(X.Embarked)

LabelEncoder()

In [47]:
# fit_transform으로 하지 말라고함
X['new_embarked']=le_embarked.transform(X.Embarked)    

In [45]:
le.transform(np.array(['S']))

array([2])

In [48]:
X.drop(['Embarked'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Sex           891 non-null    object 
 3   Age           891 non-null    float64
 4   SibSp         891 non-null    int64  
 5   Parch         891 non-null    int64  
 6   Ticket        891 non-null    object 
 7   Fare          891 non-null    float64
 8   new_cabin     891 non-null    object 
 9   new_embarked  891 non-null    int32  
dtypes: float64(2), int32(1), int64(3), object(4)
memory usage: 66.3+ KB


In [ ]:
## object들 인코딩

In [49]:
# Sex는 0,1 둘뿐이므로 np.where로 하면됨
X['gender']=np.where(X.Sex=='male',0,1)

In [50]:
X.drop(['Sex'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Ticket        891 non-null    object 
 6   Fare          891 non-null    float64
 7   new_cabin     891 non-null    object 
 8   new_embarked  891 non-null    int32  
 9   gender        891 non-null    int32  
dtypes: float64(2), int32(2), int64(3), object(3)
memory usage: 62.8+ KB


In [51]:
le_cabin=LabelEncoder()
le_cabin.fit(X.new_cabin)
X.new_cabin=le_cabin.transform(X.new_cabin)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Ticket        891 non-null    object 
 6   Fare          891 non-null    float64
 7   new_cabin     891 non-null    int32  
 8   new_embarked  891 non-null    int32  
 9   gender        891 non-null    int32  
dtypes: float64(2), int32(3), int64(3), object(2)
memory usage: 59.3+ KB


In [53]:
# 티켓은 종류가 너무 많음-->날림
X.Ticket.value_counts().size

681

In [54]:
X.drop(['Ticket'],axis='columns',inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Name          891 non-null    object 
 2   Age           891 non-null    float64
 3   SibSp         891 non-null    int64  
 4   Parch         891 non-null    int64  
 5   Fare          891 non-null    float64
 6   new_cabin     891 non-null    int32  
 7   new_embarked  891 non-null    int32  
 8   gender        891 non-null    int32  
dtypes: float64(2), int32(3), int64(3), object(1)
memory usage: 52.3+ KB
